In [1]:
# Create new features, focusing on high ROI features. 

In [2]:
# Features I already have _______________________________________
# - rewards: n, min, max, intn shipping
# - n creator backed
# - title char len
# - blurb char len
# - launch time (in Unix time)
# - project duration
# - goal

In [11]:
# I know how to do this _______________________
# - day of the week
# - day of the month -- 1st/15st/30st
# - month

In [12]:
# Next ________________________________________
# - time between launch and ship of rewards -- most recent. 

In [10]:
# To add _______________________________________
# - title word len
# - blurb word len
# - body word length
# - readablity 
# - n misspellings
# - faq n questions/word len
# - risks and rewards word len
# - n pictures
# - other reward info

In [60]:
from bs4 import BeautifulSoup
import pprint
import pymysql as mdb
import re
import pandas as pd
import numpy as np
from unidecode import unidecode
import json
import time

In [137]:
from extract_kickstarter_features import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
con = mdb.connect('localhost','root','hobbes','kickstarter')

In [6]:
with con:
    
    cur = con.cursor()
    cur.execute("SHOW TABLES")
    
    rows = cur.fetchall()
    for row in rows:
        print row

('Creator',)
('Project_info',)
('Rewards',)
('Scraped_pages',)


In [7]:
with con:
    cur = con.cursor()
    cur.execute("SELECT Id FROM Project_info \
                WHERE Category_slug LIKE 'tech%'")
    ids = cur.fetchall()
    cur.close()

In [8]:
ids_array = np.asarray(ids)

In [13]:
np.shape(ids_array)

(9351, 1)

In [18]:
ids_array[0]

array([7121])

In [19]:
with con:
    cur = con.cursor()
    cur.execute("SELECT * FROM Project_info \
                WHERE Id = %s",[7121])
    rows = cur.fetchall()
    cur.close()

In [20]:
print rows

((7121, 'SnapJet', 'https://www.kickstarter.com/profile/snapjet', 'USD', 1117814212, 'Gadgets', 'technology/gadgets', 155000, 125435, 'live', 'US', 'huntington-beach-ca', 'Huntington Beach, CA', 'A high quality open-source photo printer that seamlessly interfaces with smartphones. No wires. No apps. Just beautiful images.', 879, 'SnapJet: Turn your smartphone into a polaroid film printer!', 'https://www.kickstarter.com/projects/snapjet/snapjet-a-slim-portable-open-source-instant-film-p?ref=category', 1416427327, 1419049800),)


In [21]:
unix_t_start = 1416427327

In [22]:
# now make time features

In [23]:
# I know how to do this _______________________
# - day of the week
# - day of the month -- 1st/15st/30st
# - month

In [29]:

t_str = time.ctime(unix_t_start)
print t_str

Wed Nov 19 15:02:07 2014


In [31]:
weekday, month, day, day_t, year = t_str.split()

In [32]:
print weekday, month, day, day_t, year

Wed Nov 19 15:02:07 2014


In [41]:
print day_t[:2]

15


In [43]:
# use weekday, month, day, day_t -- just hour. done.
#  -- just use ctime -- local time, but NY is good. 
#     Kickstarter is based here.
#
# could also get day of the year, but I don't want that.

In [34]:
# next 
# convert day_t into a number
# map Nov 1 2014 into unix time

In [55]:
a = time.localtime(unix_t_start)  #weekdays are index 0=Monday

In [56]:
print a

time.struct_time(tm_year=2014, tm_mon=11, tm_mday=19, tm_hour=15, tm_min=2, tm_sec=7, tm_wday=2, tm_yday=323, tm_isdst=0)


In [59]:
# now convert 'Jan 2011' into Jan 1 2011 unix time.

In [61]:
t_str = time.ctime(unix_t_start)
print t_str

Wed Nov 19 15:02:07 2014


In [65]:
print time.mktime((2014,11,19,0,0,0,0,0,0)) # year, month, day, h,m,s

1416373200.0


In [67]:
print unix_t_start-1416373200

54127


In [68]:
54127/(60*60)

15

In [73]:
import calendar
print calendar.month_abbr[1]

Jan


In [74]:
np.where(calendar.month_abbr == 'Jan')

(array([], dtype=int64),)

In [84]:
import datetime
dt = datetime.datetime(2012, 1, 10)

In [85]:
print dt

2012-01-10 00:00:00


In [86]:
timetuple = dt.utctimetuple()
print timetuple

time.struct_time(tm_year=2012, tm_mon=1, tm_mday=10, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=1, tm_yday=10, tm_isdst=0)


In [87]:
print time.mktime(timetuple)

1326171600.0


In [88]:
reward_ship_list = ["Jan 2015", "Feb 2015", "Dec 2014", "Dec 2014", "Feb 2015", "Dec 2014", "Feb 2015", "Dec 2014", "Dec 2014", "Feb 2015", "Feb 2015", "Feb 2015", "Dec 2014", "Feb 2015", "Dec 2014", "Jun 2015", "Dec 2014", "Apr 2015", "Apr 2015"]

In [89]:
print reward_ship_list

['Jan 2015', 'Feb 2015', 'Dec 2014', 'Dec 2014', 'Feb 2015', 'Dec 2014', 'Feb 2015', 'Dec 2014', 'Dec 2014', 'Feb 2015', 'Feb 2015', 'Feb 2015', 'Dec 2014', 'Feb 2015', 'Dec 2014', 'Jun 2015', 'Dec 2014', 'Apr 2015', 'Apr 2015']


In [123]:
# just convert all to unix time
tech_df = pd.read_pickle('tech_df.save')

In [124]:
tech_df.head()

,id,pledged,goal,n rewards,min reward,max reward,intn ship,n creator backed,title n chars,blurb n chars,launch time,campaign dur
0,8018,90840,20000,13,1,10000,1,15,29,109,1391619643,2592000
1,8019,55833,50000,12,10,10000,1,1,33,105,1413818746,2595600
2,8020,6804,5000,10,1,5000,-1,12,18,129,1361591034,5184000
3,8021,313218,5000,14,10,1000,-1,24,53,95,1344533619,2715981
4,8022,74446,10000,9,9,300,1,0,22,122,1393448405,2627995


In [183]:
subset_n = np.size(tech_df['id'])
print subset_n

6633


In [201]:
# make new time predictors
# calc median time to ship
tech_to_ship_day_df = pd.DataFrame(index=range(subset_n),columns=['id','days to ship'],dtype=int)
tech_time_df = pd.DataFrame(index=range(subset_n),columns=['id','month','month day','week day','hour'],dtype=int)

In [202]:
tech_time_df.head()

,id,month,month day,week day,hour
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN


In [190]:
con = mdb.connect('localhost','root','hobbes','kickstarter')

In [212]:
with con:
    cur = con.cursor()
    for i, Id in enumerate(tech_df['id']): #tech_df['id']:
        primary_key = int(Id)
        cur.execute("SELECT Reward_expected FROM Rewards WHERE Id = %s",primary_key)
        rows = cur.fetchall()
        reward_ship_dates = rows[0][0]
        
        
        this_launch_t = tech_df.ix[i,'launch time']
        tech_to_ship_day_df.ix[i,'id'] = primary_key
        tech_to_ship_day_df.ix[i,'days to ship'] = \
                calc_days_to_reward(reward_ship_dates,this_launch_t)
        
        # now fill in time details
        
        tech_time_df.ix[i,:] = [primary_key] + list(get_time_features(this_launch_t))
        
    cur.close()

In [214]:
tech_to_ship_day_df.tail()

,id,days to ship
6628,16461,211
6629,16462,68
6630,16468,66
6631,16469,92
6632,16470,80


In [216]:
with con:
    cur = con.cursor()
    cur.execute("SELECT URL FROM Project_info WHERE Id = %s",16470)
    rows = cur.fetchall()
    print rows
    cur.close()

(('https://www.kickstarter.com/projects/augustgermar/anonabox-a-tor-hardware-router?ref=category',),)


In [213]:
tech_time_df.tail()

,id,month,month day,week day,hour
6628,16461,11,1,5,18
6629,16462,9,23,1,18
6630,16468,9,25,3,19
6631,16469,8,30,5,19
6632,16470,10,12,6,20


In [217]:
tech_df['days to ship'] = tech_to_ship_day_df['days to ship']

In [218]:
tech_df['month'] = tech_time_df['month']
tech_df['month day'] = tech_time_df['month day']
tech_df['week day'] = tech_time_df['week day']
tech_df['hour'] = tech_time_df['hour']

In [220]:
tech_df['campaign_dur'] = tech_df['campaign dur']/86400

In [222]:
tech_df.to_pickle('tech_df.save')

In [223]:
tech_df.head()

,id,pledged,goal,n rewards,min reward,max reward,intn ship,n creator backed,title n chars,blurb n chars,launch time,campaign dur,days to ship,month,month day,week day,hour,campaign_dur
0,8018,90840,20000,13,1,10000,1,15,29,109,1391619643,2592000,176,2,5,2,12,30
1,8019,55833,50000,12,10,10000,1,1,33,105,1413818746,2595600,193,10,20,0,11,30.04167
2,8020,6804,5000,10,1,5000,-1,12,18,129,1361591034,5184000,67,2,22,4,22,60
3,8021,313218,5000,14,10,1000,-1,24,53,95,1344533619,2715981,83,8,9,3,13,31.43497
4,8022,74446,10000,9,9,300,1,0,22,122,1393448405,2627995,124,2,26,2,16,30.41661
